In [1]:
from Code.preprocess import *
from Code.moransI import *
from Code.plotting import *

/athena/tilgnerlab/scratch/lim4020/2022_08_15_PSIprediction/envs/spateo/lib/python3.9/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


### Construct isoform matrix with relative expression

The create isoform matrix saves a sparse matrix, and corresponding index and column labels to the output folder. 

NOTE: this is the same step as we do before plotting the results, you only have to do this once

In [2]:
fn_allinfo = 'Data/allinfo_ds.filtered.labeled.gz'
fn_CIDmap = 'Data/sample1_barcodeToPos.CellID_ds.tsv.gz'
fn_adata = 'Data/sample1_cellbin_adjusted.h5ad'
output = 'Data/isoform_matrix'

create_isoform_matrix(fn_allinfo,
                      fn_CIDmap,
                      fn_adata,
                      output)

Potentially interesting isoforms (total): 145
Potentially interesting isoforms (novel): 44


  0%|          | 0/198 [00:00<?, ?it/s]

In [3]:
x, labels = sparse2df(input_dir = output)

### Calculate Moran's I 

For all cells ('All') and for the four major cell types. 

The running time mainly depends on the number of cells, number of tested isoforms, and number of permutations. Doing 1e6 permutations will take ~10min for this demo dataset.

The moran's I function returns 2 dataframes: 
- pval: the uncorrected p-values 
- qval: Benjamini-Yekutieli corrected values

NaN indicates that a specific isoform-celltype combination did not fulfill the testing criteria (e.g. not enough cells).

In [4]:
pval, qval = moransI(x=x,
                    labels=labels, 
                    celltypes=['All','ExciteNeuron','InhibNeuron','Astro','Oligo'],
                    output_dir='Data/moransI')

  0%|          | 0/145 [00:00<?, ?it/s]

In [5]:
pval.head()

,All,ExciteNeuron,InhibNeuron,Astro,Oligo
ENSMUST00000000388.15,0.354046,0.015590,NaN,NaN,NaN
transcript4361.chr11.nnic,0.351666,0.479775,NaN,NaN,NaN
transcript4375.chr11.nnic,0.054219,NaN,NaN,NaN,NaN
ENSMUST00000003027.14,0.004460,NaN,NaN,NaN,NaN
ENSMUST00000110998.9,0.030400,NaN,NaN,NaN,NaN


In [6]:
qval.head()

,All,ExciteNeuron,InhibNeuron,Astro,Oligo
ENSMUST00000000388.15,1.000000,0.596912,NaN,NaN,NaN
transcript4361.chr11.nnic,1.000000,1.000000,NaN,NaN,NaN
transcript4375.chr11.nnic,1.000000,NaN,NaN,NaN,NaN
ENSMUST00000003027.14,0.160831,NaN,NaN,NaN,NaN
ENSMUST00000110998.9,0.707255,NaN,NaN,NaN,NaN


Number of significant isoforms per celltype

In [7]:
(qval <= 0.05).sum()

All             17
ExciteNeuron     7
InhibNeuron      2
Astro            2
Oligo            0
dtype: int64

### Cell-type constrained permutations

For the significant isoforms using all cells, we run Moran's I with cell-type constrained permutations. This way we can check whether significance is caused by one or more cell types causing splicing changes (p-value will still be significant) or because of different cell-type proportions between the regions (p-value won't be significant anymore).

This function is significantly slower, so we only run it on the significant isoforms. This will take ~1h to run on the demo dataset.

In [8]:
iso_sign = qval.index[qval['All'] <= 0.05]

In [ ]:
res = moransI_ctperm(x=x, 
                     labels=labels, 
                     variables=iso_sign)

  0%|          | 0/17 [00:00<?, ?it/s]

In [ ]:
res.head()

In [ ]:
import seaborn as sns 
from matplotlib import pyplot as plt

g = sns.jointplot(
    x=-np.log10(res['Original']),
    y=-np.log10(res['New']),
    marginal_kws={"binwidth": 0.1, "binrange": [0.05, 4.05]},
    height=2.5, s=5,
    ratio=4
)

# Add x = y line
lims = [
    np.min([g.ax_joint.get_xlim(), g.ax_joint.get_ylim()]),  # min of both axes
    np.max([g.ax_joint.get_xlim(), g.ax_joint.get_ylim()])   # max of both axes
]
g.ax_joint.plot(lims, lims, '--', color='gray', linewidth=0.5)  # diagonal line
g.ax_joint.set_xlim(lims)
g.ax_joint.set_ylim(lims)

# Set axis labels
g.set_axis_labels('Normal permutations \n-log10(p-value)',
                  'Cell-type constrained \n-log10(p-value)')
plt.show()

For most isoforms, the p-value of the normal and cell-type constrained permutations are very similar. For some, however, we see a big difference. Let's plot an example of both.

In [ ]:
res.sort_values('New')

In [ ]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = 553190400

im_reg = Image.open('Data/Sample1_ssDNA_regist.tif')
imarray = np.array(im_reg)

In [ ]:
xlim1=5000
xlim2=17000
ylim1=4450
ylim2=18000

For example, within Snap25 has the lowest p-value possible using both normal and cell-type constrained permutations and we see spatial variation within excitatory and inhibitory neurons

In [ ]:
isoform = 'ENSMUST00000028727.11'

res.loc[isoform]

In [ ]:
spatial_hexplot(x=x, 
                labels=labels, 
                imarray=imarray,
                varName=isoform, 
                celltype='', 
                hexsize=350,
                plot_lim=[xlim1, xlim2, ylim1, ylim2])
plt.show()

for ct in ['ExciteNeuron', 'InhibNeuron', 'Oligo', 'Astro']:
    spatial_hexplot(x=x, 
                    labels=labels, 
                    imarray=imarray,
                    varName=isoform, 
                    celltype=ct, 
                    hexsize=350,
                    plot_lim=[xlim1, xlim2, ylim1, ylim2])
    plt.show()


A counter example is an isoform of Apbb1. The spatial pattern using all cells is just caused by differences between excitatory neurons (which show some expression of this isoform) compared to the other cell types.

In [ ]:
isoform = 'ENSMUST00000188368.7'

res.loc[isoform]

In [ ]:
spatial_hexplot(x=x, 
                labels=labels, 
                imarray=imarray,
                varName=isoform, 
                celltype='', 
                hexsize=350,
                plot_lim=[xlim1, xlim2, ylim1, ylim2])
plt.show()

for ct in ['ExciteNeuron', 'InhibNeuron', 'Oligo', 'Astro']:
    spatial_hexplot(x=x, 
                    labels=labels, 
                    imarray=imarray,
                    varName=isoform, 
                    celltype=ct, 
                    hexsize=350,
                    plot_lim=[xlim1, xlim2, ylim1, ylim2])
    plt.show()
